# 4. Inference

In [25]:
import sys, os
import pandas as pd
sys.path.append("../../../../")

from Clust.clust.ML.common import ML_pipeline, tool
from Clust.clust.ML.tool import scaler as ml_scaler
from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient
db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


## 1. Data Preparation

In [26]:
app_name= "energy" # "Hs2SwineFarmWithWeatherTime", "energy"
model_method = 'GRU_rg'
model_clean = True
step = 'test'
bucket_name = 'integration'
# data_clean_level=0

In [34]:
model_name = 'energy_GRU_rg_False'

collection_name = 'meta'
db_name = 'model'
model_meta_list= mongo_client.get_document_by_json(db_name, collection_name, {'modelName': model_name})
model_meta = model_meta_list[0] ## ??

In [36]:
model_meta

{'trainDataInfo': {'bucket_name': 'integration',
  'collection_name': 'regression_energy',
  'ms_name': 'regression_energy_cleanLevel4_trainX',
  'ingestion_type': 'multiple_ms_by_time',
  'ingestion_param': {'ms_list_info': [['life_indoor_environment',
     'humidityTrain_10min'],
    ['life_indoor_environment', 'temperatureTrain_10min'],
    ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']],
   'start_time': '2016-01-11',
   'end_time': '2016-04-15'},
  'processing_type': 'step_3',
  'process_param': {'refine_param': {'removeDuplication': {'flag': True},
    'staticFrequency': {'flag': True, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': True},
    'unCertainErrorToNaN': {'flag': True,
     'param': {'outlierDetectorConfig': [{'algorithm': 'IQR',
        'percentile': 99,
        'alg_parameter': {'weight': 100}}]}}},
   'imputation_param': {'flag': True,
    'imputation_method': [{'min': 0,
      'max': 2,
      'method': 'linear',
     

### 1-2. Data Ingestion
#### 1-2-1. X-y Data Ingestion pipeline

In [37]:
data_clean_level = 4

model_purpose = model_meta['modelPurpose']
dataset_name = model_purpose + '_' + app_name  
scaler_param= model_meta['scalerParam']
data_name_X = dataset_name + '_cleanLevel' + str(data_clean_level)+'_'+step+'X'
data_name_y = dataset_name+'_cleanLevel' + str(data_clean_level)+'_'+ step+'y'
data_y_flag = model_meta['data_y_flag']

In [38]:
feature_X_list = model_meta['featureXList']
feature_y_list = model_meta["featureyList"]
model_file_path = model_meta['files']['modelFile']["filePath"]

dataX = db_client.get_data(bucket_name, data_name_X)

#### 1-2-2. Random Nan Insert (Test)
#### 1-2-3. Data scaling

In [39]:
X_scaler_file_path = model_meta['files']['XScalerFile']["filePath"]
y_scaler_file_path = model_meta['files']['yScalerFile']["filePath"]
y_scaler_file_path

'./scaler/regression_energy_cleanLevel4_trainy/minmax/f69156750a210491ffd4a67b605bc88b/scaler.pkl'

In [40]:
scaler_param = model_meta["scalerParam"]
infer_X, scaler_X = ml_scaler.get_scaled_test_data(dataX, X_scaler_file_path, scaler_param)
scaler_y = ml_scaler.get_scaler_file(y_scaler_file_path)

#### 1-2-2. Random Nan Insert (Test X)
#### 1-2-3. Data scaling

In [41]:
# split_mode = model_meta['dataSplitMode']
# transformParameter = model_meta["transformParameter"]
# test_X_array, test_y_array = ML_pipeline.transform_data_by_split_mode(split_mode, transformParameter, test_X, test_y)

In [42]:
from Clust.clust.transformation.type.DFToNPArray import trans_df_to_np_inf
infer_X = trans_df_to_np_inf(infer_X)

../../../../Clust/clust/transformation/type/DFToNPArray.py:307: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfX_partial = dfX[startDate:endDate]


In [43]:
modelParameter = model_meta["modelParameter"]

inference_parameter = {
    'device': 'cpu',
    'batch_size': model_meta["trainParameter"]['batch_size']
}

In [44]:
from Clust.clust.ML.regression.inference import RegressionInference as RI

In [45]:
def CLUST_regression_inference(infer_X,inference_parameter, model_method, model_file_path, modelParameter):
    from Clust.clust.ML.regression.inference import RegressionInference as RI
    ri = RI()
    ri.set_param(inference_parameter)
    ri.set_model(model_method, model_file_path, modelParameter)
    ri.set_data(infer_X)
    preds = ri.inference()
    
    return preds

In [46]:
model_file_path

'./Models/GRU_rg/energy_GRU_rg_False/regression_energy_cleanLevel4_trainX/c6f2ca56e27be1937ecf1e173440959e/model.pkl'

In [47]:
preds = CLUST_regression_inference(infer_X,inference_parameter, model_method, model_file_path, modelParameter)

inference data shape: torch.Size([42, 24, 144])

Start inference

** Dimension of result for inference dataset = (48,)


In [66]:
# for regression
target = feature_y_list[0]
if app_name == "energy":
    if scaler_param =='scale':
        base_df_for_inverse = pd.DataFrame(columns=feature_y_list, index=range(len(preds)))
        print(base_df_for_inverse)
        base_df_for_inverse[target] = preds
        print(base_df_for_inverse)
        prediction_result = pd.DataFrame(scaler_y.inverse_transform(base_df_for_inverse), columns=feature_y_list, index=base_df_for_inverse.index)
    else:
        prediction_result = pd.DataFrame(data={"value": preds}, index=range(len(preds)))

# for forecast
elif app_name == "Hs2SwineFarmWithWeatherTime":
    if scaler_param =='scale':
        base_df_for_inverse = pd.DataFrame(columns=feature_list, index=range(len(preds)))
        base_df_for_inverse[target] = preds
        inverse_result = pd.DataFrame(scaler_X.inverse_transform(base_df_for_inverse), columns=feature_list, index=base_df_for_inverse.index)
        target_data = inverse_result[target]
        prediction_result = pd.DataFrame(data={target: target_data}, index=range(len(preds)))
    else:
        prediction_result = pd.DataFrame(data={target: preds}, index=range(len(preds)))

   value
0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
15   NaN
16   NaN
17   NaN
18   NaN
19   NaN
20   NaN
21   NaN
22   NaN
23   NaN
24   NaN
25   NaN
26   NaN
27   NaN
28   NaN
29   NaN
30   NaN
31   NaN
32   NaN
33   NaN
34   NaN
35   NaN
36   NaN
37   NaN
38   NaN
39   NaN
40   NaN
41   NaN
42   NaN
43   NaN
44   NaN
45   NaN
46   NaN
47   NaN
       value
0   0.413548
1   0.447503
2   0.279406
3   0.326261
4   0.263888
5   0.427857
6   0.374767
7   0.576168
8   0.414155
9   0.445514
10  0.325314
11  0.315227
12  0.447088
13  0.294879
14  0.324679
15  0.400221
16  0.366732
17  0.433702
18  0.343399
19  0.478326
20  0.322100
21  0.285789
22  0.324141
23  0.251547
24  0.301857
25  0.411362
26  0.374612
27  0.411399
28  0.377005
29  0.361636
30  0.616859
31  0.361342
32  0.370216
33  0.322997
34  0.342940
35  0.477962
36  0.437472
37  0.215889
38  0.350974
39  0.405665
40  0.637666
41  0.254668


In [68]:
prediction_result

,value
0,13.999688
1,14.710714
2,11.190770
3,12.171912
4,10.865810
5,14.299318
6,13.187617
7,17.404964
8,14.012404
9,14.669057
